# Servier - List
## Beachten
* Gewisse Spalten sind mit Linie, gewisse nicht. Deshalb Spalten manuell definiert
* Hat Rundungsdifferenzen

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [6]:
# Read pdf into DataFrame
columns = [80.5, 198.3, 245.7, 270.9, 478.3, 583.5, 609.8, 641.4, 667.7, 690.9, 721.4, 754]
df = tabula.read_pdf("EFPIA_2019_-_Divulgation_ALL_V2.pdf", pages='all', lattice=False, columns=columns, guess=False)

## Format Table

In [16]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']


#Copy rows with no name with value from the previous  row (was multiple row)
for index, row in df_export[df_export.name.isna() & df_export.total.notna()].iterrows():
    if index > 0:
        df_export.loc[index, 'name'] = df_export.loc[index - 1, 'name']
        df_export.loc[index, 'location'] = df_export.loc[index - 1, 'location']
        df_export.loc[index, 'country'] = df_export.loc[index - 1, 'country']
        df_export.loc[index, 'address'] = df_export.loc[index - 1, 'address']    

#Add Type
index_hco = df_export[df_export['name'].str.contains("ANDERE, OBEN NICHT GENANNTE", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')        

        
#Remove empty rows
df_export = df_export.dropna(subset=['address'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')

#Remove by string
df_export = df_export[~df_export['name'].str.contains('Art. 3', na=False)]
df_export = df_export[~df_export['name'].str.contains('1. Full name', na=False)]

#Remove empties
df_export.drop(columns=['empty0'], inplace=True)

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Cleanup Numbers and sum
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

#Export PLZ
df_export['plz'] = df_export['location'].str.extract(r'(^\S*)', expand=True)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'servier')

Total nicht Summe der Werte
saved


In [17]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx', open=True)